In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
print(train.shape)

train[['C-deck', 'C-num', 'C-side']] = train['Cabin'].str.split('/', expand=True)

train[['gggg','pp']] = train['PassengerId'].str.split('_', expand=True).astype(int)

train1 = train
train1['Count'] = 0
for i in set(train1['pp']):
    for index, row in train1.iterrows():
        if row['pp'] == i:
            train1.loc[train1['gggg'] == row['gggg'], 'Count'] = i

train1.head()

In [ ]:
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
print(test.shape)

test[['C-deck', 'C-num', 'C-side']] = test['Cabin'].str.split('/', expand=True)

test[['gggg','pp']] = test['PassengerId'].str.split('_', expand=True).astype(int)

test1 = test 
test1['Count'] = 0
for i in set(test1['pp']):
    for index, row in test1.iterrows():
        if row['pp'] == i:
            test1.loc[test1['gggg'] == row['gggg'], 'Count'] = i
                
test1.head(5)

In [ ]:
#Check and print if any group(PassengerId) has been split in train and test sets 
for i in set(test1['gggg']):
    for j in set(train1['gggg']):
        if i == j:
            print(test1['gggg'])

In [ ]:
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
submission.head(2)

In [ ]:
print(train1.isna().sum().sort_values(ascending = False),'\n')
print(test1.isna().sum().sort_values(ascending = False))

In [ ]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp = IterativeImputer(random_state = 0)
# imp2 = IterativeImputer(random_state = 0)

# imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService", "HomePlanet", "CryoSleep", "Destination"]
# imp.fit(train1[imputer_cols])
# imp2.fit(test1[imputer_cols])


# train1[imputer_cols] = imp.transform(train1[imputer_cols])
# test1[imputer_cols] = imp2.transform(test1[imputer_cols])

In [ ]:
from sklearn.impute import KNNImputer

imp = KNNImputer()
imp2 = KNNImputer()

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService", "HomePlanet", "CryoSleep", "Destination"]

imp.fit(train1[imputer_cols])
imp2.fit(test1[imputer_cols])

train1[imputer_cols] = imp.transform(train1[imputer_cols])
test1[imputer_cols] = imp2.transform(test1[imputer_cols])

In [ ]:
from sklearn.impute import SimpleImputer

# imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService", "HomePlanet", "CryoSleep", "Destination"]
# STRATEGY = 'most_frequent'
# imputer = SimpleImputer(strategy=STRATEGY)
# imputer.fit(train1[imputer_cols])
# train1[imputer_cols] = imputer.transform(train1[imputer_cols])

# imputer2 = SimpleImputer(strategy=STRATEGY)
# imputer2.fit(test1[imputer_cols])
# test1[imputer_cols] = imputer2.transform(test1[imputer_cols])

train1["HomePlanet"].fillna('Z', inplace=True)
test1["HomePlanet"].fillna('Z', inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP", "C-deck", "C-num", "C-side"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train2 ,test2 = label_encoder(train1,test1 ,label_cols)

In [ ]:
train2.head()

In [ ]:
train3 = train2.drop(["PassengerId","Name" ,"Cabin", "gggg", "pp"] , axis = 1)
test3 = test2.drop(["PassengerId","Name" ,"Cabin", "gggg", "pp"] , axis = 1)

train3['Total_Spend'] = train3['RoomService']+train3['FoodCourt']+train3['ShoppingMall']+train3['Spa']+train3['VRDeck']
test3['Total_Spend'] = test3['RoomService']+test3['FoodCourt']+test3['ShoppingMall']+test3['Spa']+test3['VRDeck']

In [ ]:
X = train3.drop('Transported' , axis =1 )
y = train3['Transported']

In [ ]:
X = X.astype('float32')
X.head()

In [ ]:
#Normalise the data
from sklearn.preprocessing import StandardScaler

scale = StandardScaler().fit(X)

X_train_scaled = X #scale.transform(X)
X_test_scaled = test3 #scale.transform(test3)

In [ ]:
y=y.astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y, test_size=0.1)

In [ ]:
from catboost import CatBoostClassifier
model=CatBoostClassifier(iterations=10000,eval_metric='Accuracy',verbose=1000)

#model.fit(X_train,y_train,eval_set=(X_val,y_val),use_best_model=True)
model.fit(X_train_scaled,y)
print(model.get_best_iteration())
print(model.random_seed_)
print(model.learning_rate_)

In [ ]:
# from sklearn.metrics import accuracy_score
# out=model.predict(X_val)
# out_t=model.predict(X_train)

# print(accuracy_score(y_train.values,out_t))
# print(accuracy_score(y_val.values,out))

In [ ]:
ans=model.predict(X_test_scaled)
ans=ans.astype(bool)

In [ ]:
submission['Transported']=ans
submission.to_csv('submission.csv', index=False)
submission.head(10)